In [ ]:
import numpy as np
import os
from tensorflow.keras.applications import MobileNetV3Small, ResNet50
from tensorflow.keras.applications.efficientnet import EfficientNetB3
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
train_dir = '/kaggle/input/original-augmentation-image-processing/split/train'
test_dir = '/kaggle/input/original-augmentation-image-processing/split/test'

In [ ]:
batch_size=256

In [ ]:
train_data = image_dataset_from_directory(
    train_dir,
    label_mode="categorical",
    image_size=(224, 224),
    batch_size=batch_size,
    shuffle=True,
    seed=42
)

test_data = image_dataset_from_directory(
    test_dir,
    label_mode="categorical",
    image_size=(224, 224),
    batch_size=batch_size,
    shuffle=True,
    seed=42
)

In [ ]:
model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
for layer in model.layers:
    layer.trainable = False
model.summary()

In [ ]:
train_features = []
train_labels = []

for batch_images, batch_labels in train_data:
    features_batch = model.predict(batch_images)
    train_features.append(features_batch)
    train_labels.append(batch_labels)

train_features = np.concatenate(train_features)
train_labels = np.concatenate(train_labels)
train_features = train_features.reshape((train_features.shape[0], -1))

In [ ]:
test_features = []
test_labels = []

for batch_images, batch_labels in test_data:
    features_batch = model.predict(batch_images)
    test_features.append(features_batch)
    test_labels.append(batch_labels)

test_features = np.concatenate(test_features)
test_labels = np.concatenate(test_labels)
test_features = test_features.reshape((test_features.shape[0], -1))

In [ ]:
"""Random Forest"""
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(train_features, np.argmax(train_labels, axis=1))
train_predictions_rf = rf_classifier.predict(train_features)
test_predictions_rf = rf_classifier.predict(test_features)
print (f"Training accuracy (RF) =  {accuracy_score(np.argmax(train_labels, axis=1), train_predictions_rf)}")
print (f"Testing accuracy (RF) =  {accuracy_score(np.argmax(test_labels, axis=1), test_predictions_rf)}")

In [ ]:
"""SVM"""
svm_classifier = SVC(kernel='linear', C=1.0, random_state=42)
svm_classifier.fit(train_features, np.argmax(train_labels, axis=1))
train_predictions_svm = svm_classifier.predict(train_features)
test_predictions_svm = svm_classifier.predict(test_features)
print (f"Training accuracy (SVM) = {accuracy_score(np.argmax(train_labels, axis=1), train_predictions_svm)}")
print (f"Testing accuracy (SVM) = {accuracy_score(np.argmax(test_labels, axis=1), test_predictions_svm)}")